In [1]:
import pandas as pd
import scipy as scp
import statistics as st
import numpy as np


In [2]:
Covid=pd.read_csv("Corona_Updated.csv")
Covid.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Temprature,Humidity,Temp_Cat,Humid_Cat
0,Hubei,Mainland China,2020-03-10T15:13:05,67760,3024,47743,30.9756,112.2707,12.5,86,1,1
1,NaN,Italy,2020-03-10T17:53:02,10149,631,724,43.0000,12.0000,12.9,64,1,1
2,NaN,Iran (Islamic Republic of),2020-03-10T19:13:20,8042,291,2731,32.0000,53.0000,11.9,9,0,0
3,NaN,Republic of Korea,2020-03-10T19:13:20,7513,54,247,36.0000,128.0000,4.9,41,0,0
4,NaN,France,2020-03-10T18:53:02,1784,33,12,47.0000,2.0000,11.9,93,0,0


## Problem Statement

In [3]:
# Ojective:

# A common perception about COVID-19 is that Warm Climate is more resistant to the corona outbreak 
# and we need to verify this using Hypothesis Testing. 


#### Null and Alternate Hypothesis

In [4]:
# Based on the objective the 
# Null Hypothesis(H0)= Temperature does not affect covid outbreak
# Alternate Hypothesis(HA)= Temperature does affect covid outbreak

#### Data Preparation

In [5]:
#Note: We are considering Temperature below 24 as cold climate and above 24 as hot climate in our dataset

In [6]:
Covid.describe()
# Covid.isnull().sum()

,Confirmed,Deaths,Recovered,Latitude,Longitude,Temprature,Humidity,Temp_Cat,Humid_Cat
count,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000
mean,575.640777,20.689320,312.640777,31.184989,11.752030,12.161165,67.728155,0.470874,0.470874
std,4822.697784,215.794845,3332.764713,21.305149,84.576291,10.229763,21.780588,0.500367,0.500367
min,0.000000,0.000000,0.000000,-41.454500,-157.498300,-21.900000,6.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,25.069200,-74.841325,6.100000,55.000000,0.000000,0.000000
50%,12.000000,0.000000,0.000000,36.030550,15.234250,11.750000,73.000000,0.000000,0.000000
75%,75.750000,1.000000,4.000000,43.870250,101.363375,20.375000,84.000000,1.000000,1.000000
max,67760.000000,3024.000000,47743.000000,64.963100,174.886000,33.100000,98.000000,1.000000,1.000000


In [7]:
Covid['Temp_Cat'] = Covid['Temprature'].apply(lambda x : 0 if x < 24 else 1)

In [8]:
print(Covid.loc[Covid['Temp_Cat']==1].count())
print(Covid.loc[Covid['Temp_Cat']==0].count())


Province/State     4
Country/Region    31
Last Update       31
Confirmed         31
Deaths            31
Recovered         31
Latitude          31
Longitude         31
Temprature        31
Humidity          31
Temp_Cat          31
Humid_Cat         31
dtype: int64
Province/State     95
Country/Region    175
Last Update       175
Confirmed         175
Deaths            175
Recovered         175
Latitude          175
Longitude         175
Temprature        175
Humidity          175
Temp_Cat          175
Humid_Cat         175
dtype: int64


In [9]:
Covid_t = Covid[['Confirmed', 'Temp_Cat']]

In [10]:
Covid_t

,Confirmed,Temp_Cat
0,67760,0
1,10149,0
2,8042,0
3,7513,0
4,1784,0
...,...,...
201,0,0
202,0,0
203,0,0
204,0,0


### Implementing the Two-Sample Z test for the dataset.

In [12]:
d1= Covid_t[Covid_t['Temp_Cat']==1]['Confirmed']
d2= Covid_t[Covid_t['Temp_Cat']==0]['Confirmed']

# storing confirmed cases of hot and cold regions in different series object

In [13]:
print(d1.count(),d2.count())


31 175


##### Sample Mean 1 and Sample Mean 2 

In [14]:
m1,m2 = d1.mean(),d2.mean()  # finding mean confirmed cases of hot and cold regions

In [15]:
print(m1,m2)

26.548387096774192 672.9085714285715


##### Standard Deviation 1 and Standard Deviation 2

In [16]:
sd1,sd2 = d1.std(),d2.std()   # finding Standard deviation in confirmed cases of hot and cold regions

print(sd1,sd2)

##### Sample Size 1 and Sample Size 2

In [18]:
n1,n2 = d1.shape[0],d2.shape[0]  #finding the number of samples of confimed cases in hot and cold regions

In [19]:
print(n1,n2)

31 175


##### Population mean 1 and 2

In [67]:
# mudiff= mu1-mu2 as
# The population means of two groups are not significantly different in a 2 sample Z test as per null hypothesis.
mudiff=0

#### Hypothesis Test using Two Sample Z-Test

In [85]:
def TwoSampleZTest(X1,X2,mudiff,sigma1,sigma2,N1,N2):
    from scipy.stats import norm
    StdError_denom=np.sqrt(sigma1**2/N1 +sigma2**2/N2)
    z=((X1-X2)-mudiff)/StdError_denom
    pval=2*(1-norm.cdf(abs(z)))   # p value multiplied by 2 because of 2 tailed Z test
    return z,pval

# 1 - norm.cdf can be replaced in numpy using norm.sf for a better precision

In [88]:
Z,P = TwoSampleZTest(m1,m2,mudiff,sd1,sd2,n1,n2)

Z_Score= np.round(Z,8)
P_Value= np.round(P,6)

In [89]:
if P_Value <0.05:
    Hypothesis_status='Reject Null Hypothesis: Significant'
else:
    Hypothesis_status='Do not Reject Null Hypothesis: Not Significant'


#### Result

In [90]:

print(P_Value)
print(Hypothesis_status)

0.102054
Do not Reject Null Hypothesis: Not Significant


In [91]:
#chaging p-value formula by replacing norm.cdf in numpy with norm.sf

In [98]:
def TwoSampleZTest2(X1,X2,mudiff,sigma1,sigma2,N1,N2):
    from scipy.stats import norm
    StdError_denom=np.sqrt(sigma1**2/N1 +sigma2**2/N2)
    z=((X1-X2)-mudiff)/StdError_denom
    p_value = 2*(norm.sf(abs(z))) 
    return z,p_value



In [99]:
Z,P = TwoSampleZTest2(m1,m2,mudiff,sd1,sd2,n1,n2)

Z_Score= np.round(Z,8)
P_Value= np.round(P,6)

In [100]:
if P_Value <0.05:
    Hypothesis_status='Reject Null Hypothesis: Significant'
else:
    Hypothesis_status='Do not Reject Null Hypothesis: Not Significant'


In [101]:

print(P_Value)
print(Hypothesis_status)

0.102054
Do not Reject Null Hypothesis: Not Significant
